In [1]:
import os
from lca_algebraic import initProject
import pandas as pd
import numpy as np
import lca_algebraic as agb
from lca_algebraic import *
from lca_algebraic.stats import * 
import bw2data, bw2io
import bw2calc
from premise import *
import premise as prm
from premise.geomap import Geomap
import pickle
from itertools import product 
from IPython.display import display
import sys 
#from premise_gwp import add_premise_gwp

In [2]:
sys.path.append('/Users/susierwu/Dropbox/DLCA_DynCO2/pdpLCA_GT/') 
from utils import *
from utils.setup import *
from utils.setup import chosen_methods
from utils.getAct import *
from utils.utils import *
from utils.switchElec import *

modify the setup.py file for reading Parameter excel sheet. here we use 'data/p_file/p_tech_perSSP_Y.xlsx' as the technology parameter file.  
Define your chosen methods in the setup.py file. Current methods are:  [('IPCC 2021 no LT', 'climate change no LT', 'global warming potential (GWP100) no LT'), ('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')]
Mapping premise_remind_DB to SSPx:  {'SSP1-PkBudg500': 'ssp119', 'SSP1-PkBudg1150': 'ssp126', 'SSP2-Base': 'ssp245', 'SSP5-Base': 'ssp585'}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
utils.switchElec is to update an UPRs with the switch elec parameter, allow users to switch manuf. electricity grid per their defined locations - market group or grid-specific


In [3]:
#bw2data.projects
bw2data.projects.set_current('iveo_v1')
mybio = bw2data.Database("biosphere3")
initProject("iveo_v1")

Biosphere database already present!!! No setup is needed


### init project

In [4]:
pdb = bw2data.Database(PDB_NAME)
print(len(pdb), PDB_NAME)

322 iveo_Parameterized_v1


In [5]:
# use .loadParams if happy with P already defined
resetParams()
agb.loadParams()
agb.list_parameters()

## A. agb, prepare tech (LIB_NMC622 , NCA, LFP, LTO) ) UPR, copy from premise, then convert FU kg to kWh 

### A.0_parameters for LIBs (FU per kWh), sheet"V1B_bat_prmLIBs_perkWh", P group "battery": 

In [6]:
NMC622_battery_cell_density = newFloatParam(
        name="NMC622_battery_cell_density",
        default=0.29161,   
        min=0.25,
        max=0.426946201,
        distrib=stats.DistributionType.TRIANGLE,
        group="battery",
        # label =  
        unit="kWh/kg",
    )

NMC_cell_neededkg_perkWh = 1/NMC622_battery_cell_density   

"""
NMC622_cell_perkgBat_massratio = newFloatParam(
        name="NMC622_cell_perkgBat_massratio",
        default=0.6,   
        min=0.5,
        max=0.7,
        distrib=stats.DistributionType.TRIANGLE,
        group="battery",
        unit="unitless",
    )
"""

NCA_battery_cell_density = newFloatParam(
        name="NCA_battery_cell_density",
        default=0.303589,   
        min=0.25,
        max=0.444484655,
        distrib=stats.DistributionType.TRIANGLE,
        group="battery",
        unit="kWh/kg",
    )

LFP_battery_cell_density = newFloatParam(
        name="LFP_battery_cell_density",
        default=0.21054,   
        min=0.2,
        max=0.308251614,
        distrib=stats.DistributionType.TRIANGLE,
        group="battery",
        unit="kWh/kg",
    )

LTO_battery_cell_density = newFloatParam(
        name="LTO_battery_cell_density",
        default=0.21054,   
        min=0.2,
        max=0.308251614,
        distrib=stats.DistributionType.TRIANGLE,
        group="battery",
        unit="kWh/kg",
    )


"""
LTO_cell_perkgBat_massratio = newFloatParam(
        name="LTO_cell_perkgBat_massratio",
        default=0.6,   
        min=0.5,
        max=0.7,
        distrib=stats.DistributionType.TRIANGLE,
        group="battery",
        unit="unitless",
    )
"""

'\nLTO_cell_perkgBat_massratio = newFloatParam(\n        name="LTO_cell_perkgBat_massratio",\n        default=0.6,   \n        min=0.5,\n        max=0.7,\n        distrib=stats.DistributionType.TRIANGLE,\n        group="battery",\n        unit="unitless",\n    )\n'

## A.1 NMC:: add cell + BoP  -> final battery pack per kWh

In [7]:
NMC_final_allsspyear = []
for prm_name in ssp_scn:
    for year_t in ssp_yr: 
        bg_db_ssp_yr = bg_ei_ + prm_name + "_" + str(year_t)            # backgroun database (BD) to use for SSP[x], Year[t]
        copied_prm_UPR_suffix = "_from_" + prm_name + "_" + str(year_t) # copy UPR from BD to PDB for further use/modification if needed
        newUPR_suffix =  "_wh_" + prm_name + "_" + str(year_t)          # naming on any new UPRs created in PDB
        
        # NCM final battery pack : need to add cell then BoP for final 1 kg of battery pack
        BAT_NMC_battery_pack = agb.newActivity(
                db_name=PDB_NAME,
                name = "V1B_NMC622 battery_final battery pack_perkWh"  +  newUPR_suffix, loc = 'CA-QC', 
                code = "V1B_NMC622_battery_final_battery_pack_perkWh"  + newUPR_suffix, 
                unit="kWh",
                # NMC622_cell_perkgBat_massratio is assumed to be 0.6 per premise, not changing parameter
                exchanges={
                    agb.findActivity("battery cell, NMC-622", loc = "GLO", db_name = bg_db_ssp_yr):  1*0.6 * (1/NMC622_battery_cell_density	)   ,#the ratio using prm "heavy duty truck, battery electric, NMC-622 battery, 32t gross weight"     
                    agb.findActivity('battery BoP', loc = 'GLO',  db_name = bg_db_ssp_yr): 1* (1-0.6)  * (1/NMC622_battery_cell_density	)  ,   
                },
        )
        
        for exc in BAT_NMC_battery_pack.getExchange(name = "*", single=False): 
            exc['type']='technosphere'
            exc.save()
        BAT_NMC_battery_pack.setOutputAmount(1)
                
        #agb.printAct(BAT_NMC_battery_pack)
        NMC_final_allsspyear.append(BAT_NMC_battery_pack)
        print(f"end of creating a new UPR for NMC622 final battery pack for SSP {prm_name} and year {year_t}, relying on BD {bg_db_ssp_yr}")
        

Activity 'V1B_NMC622_battery_final_battery_pack_perkWh_wh__remind_SSP1-PkBudg500_2030' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for NMC622 final battery pack for SSP _remind_SSP1-PkBudg500 and year 2030, relying on BD ecoinvent_cutoff_3.9_remind_SSP1-PkBudg500_2030


Activity 'V1B_NMC622_battery_final_battery_pack_perkWh_wh__remind_SSP1-PkBudg500_2040' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for NMC622 final battery pack for SSP _remind_SSP1-PkBudg500 and year 2040, relying on BD ecoinvent_cutoff_3.9_remind_SSP1-PkBudg500_2040


Activity 'V1B_NMC622_battery_final_battery_pack_perkWh_wh__remind_SSP1-PkBudg500_2050' was already in 'iveo_Parameterized_v1'. Overwriting it


...

Activity 'V1B_NMC622_battery_final_battery_pack_perkWh_wh__remind_SSP5-Base_2030' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for NMC622 final battery pack for SSP _remind_SSP5-Base and year 2030, relying on BD ecoinvent_cutoff_3.9_remind_SSP5-Base_2030


Activity 'V1B_NMC622_battery_final_battery_pack_perkWh_wh__remind_SSP5-Base_2040' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for NMC622 final battery pack for SSP _remind_SSP5-Base and year 2040, relying on BD ecoinvent_cutoff_3.9_remind_SSP5-Base_2040
end of creating a new UPR for NMC622 final battery pack for SSP _remind_SSP5-Base and year 2050, relying on BD ecoinvent_cutoff_3.9_remind_SSP5-Base_2050


Activity 'V1B_NMC622_battery_final_battery_pack_perkWh_wh__remind_SSP5-Base_2050' was already in 'iveo_Parameterized_v1'. Overwriting it


## A.2 LFP:: final battery pack kWh
- battery pack for LFP is already prepared "market for battery, Li-ion, LFP, rechargeable, prismatic | GLO" - ... production of 1 kg of Li-ion battery pack used e.g. for mechanical drive of an electric vehicle, so no need to seperately add cell + BoP 

In [8]:
for prm_name in ssp_scn:
    for year_t in ssp_yr: 
        bg_db_ssp_yr = bg_ei_ + prm_name + "_" + str(year_t)            # backgroun database (BD) to use for SSP[x], Year[t]
        copied_prm_UPR_suffix = "_from_" + prm_name + "_" + str(year_t) # copy UPR from BD to PDB for further use/modification if needed
        newUPR_suffix =  "_wh_" + prm_name + "_" + str(year_t)          # naming on any new UPRs created in PDB
        
        # LFP final battery pack directly from premise, 
        BAT_battery_pack = agb.newActivity(
                db_name=PDB_NAME,
                name = "V1B_LFP battery_final battery pack_perkWh"  +  newUPR_suffix, loc = 'CA-QC', 
                code = "V1B_LFP_battery_final_battery_pack_perkWh"  + newUPR_suffix, 
                unit="kWh",
                exchanges={
                    agb.findActivity("market for battery, Li-ion, LFP, rechargeable, prismatic", loc = "GLO", db_name = bg_db_ssp_yr): 1 * (1/LFP_battery_cell_density)    
                },
        )
        
        for exc in BAT_battery_pack.getExchange(name = "*", single=False): 
            exc['type']='technosphere'
            exc.save()
        BAT_battery_pack.setOutputAmount(1)
                
        #agb.printAct(BAT_NMC_battery_pack)
        print(f"end of creating a new UPR for LFP final battery pack for SSP {prm_name} and year {year_t}, relying on BD {bg_db_ssp_yr}")

Activity 'V1B_LFP_battery_final_battery_pack_perkWh_wh__remind_SSP1-PkBudg500_2030' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for LFP final battery pack for SSP _remind_SSP1-PkBudg500 and year 2030, relying on BD ecoinvent_cutoff_3.9_remind_SSP1-PkBudg500_2030


Activity 'V1B_LFP_battery_final_battery_pack_perkWh_wh__remind_SSP1-PkBudg500_2040' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for LFP final battery pack for SSP _remind_SSP1-PkBudg500 and year 2040, relying on BD ecoinvent_cutoff_3.9_remind_SSP1-PkBudg500_2040


Activity 'V1B_LFP_battery_final_battery_pack_perkWh_wh__remind_SSP1-PkBudg500_2050' was already in 'iveo_Parameterized_v1'. Overwriting it


...

end of creating a new UPR for LFP final battery pack for SSP _remind_SSP5-Base and year 2030, relying on BD ecoinvent_cutoff_3.9_remind_SSP5-Base_2030


Activity 'V1B_LFP_battery_final_battery_pack_perkWh_wh__remind_SSP5-Base_2040' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for LFP final battery pack for SSP _remind_SSP5-Base and year 2040, relying on BD ecoinvent_cutoff_3.9_remind_SSP5-Base_2040


Activity 'V1B_LFP_battery_final_battery_pack_perkWh_wh__remind_SSP5-Base_2050' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for LFP final battery pack for SSP _remind_SSP5-Base and year 2050, relying on BD ecoinvent_cutoff_3.9_remind_SSP5-Base_2050


## A.3 LTO:: add cell + BoP ==> final battery pack per kWh

- similar to NMC, only "battery cell, LTO", no final bat pack like LFP, need to add in bat BoP 

In [9]:
for prm_name in ssp_scn:
    for year_t in ssp_yr: 
        bg_db_ssp_yr = bg_ei_ + prm_name + "_" + str(year_t)            # backgroun database (BD) to use for SSP[x], Year[t]
        copied_prm_UPR_suffix = "_from_" + prm_name + "_" + str(year_t) # copy UPR from BD to PDB for further use/modification if needed
        newUPR_suffix =  "_wh_" + prm_name + "_" + str(year_t)          # naming on any new UPRs created in PDB
        
        # NCM final battery pack : need to add cell then BoP for final 1 kg of battery pack
        BAT_battery_pack = agb.newActivity(
                db_name=PDB_NAME,
                name = "V1B_LTO battery_final battery pack_perkWh"  +  newUPR_suffix, loc = 'CA-QC', 
                code = "V1B_LTO_battery_final_battery_pack_perkWh"  + newUPR_suffix, 
                unit="kWh",
                #LTO_cell_perkgBat_massratio same value 0.6 as NMC622
                exchanges={
                    agb.findActivity("battery cell, LTO", loc = "GLO", db_name = bg_db_ssp_yr):  1 * 0.6  * (1/LTO_battery_cell_density)   ,#the ratio using prm "heavy duty truck, battery electric, NMC-622 battery, 32t gross weight"     
                    agb.findActivity('battery BoP', loc = 'GLO',  db_name = bg_db_ssp_yr): 1 * (1 - 0.6) * (1/LTO_battery_cell_density) ,   
                },
        )
        
        for exc in BAT_battery_pack.getExchange(name = "*", single=False): 
            exc['type']='technosphere'
            exc.save()
        BAT_battery_pack.setOutputAmount(1)
                
        print(f"end of creating a new UPR for LTO final battery pack for SSP {prm_name} and year {year_t}, relying on BD {bg_db_ssp_yr}")
        

Activity 'V1B_LTO_battery_final_battery_pack_perkWh_wh__remind_SSP1-PkBudg500_2030' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for LTO final battery pack for SSP _remind_SSP1-PkBudg500 and year 2030, relying on BD ecoinvent_cutoff_3.9_remind_SSP1-PkBudg500_2030


Activity 'V1B_LTO_battery_final_battery_pack_perkWh_wh__remind_SSP1-PkBudg500_2040' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for LTO final battery pack for SSP _remind_SSP1-PkBudg500 and year 2040, relying on BD ecoinvent_cutoff_3.9_remind_SSP1-PkBudg500_2040


Activity 'V1B_LTO_battery_final_battery_pack_perkWh_wh__remind_SSP1-PkBudg500_2050' was already in 'iveo_Parameterized_v1'. Overwriting it


...

end of creating a new UPR for LTO final battery pack for SSP _remind_SSP5-Base and year 2030, relying on BD ecoinvent_cutoff_3.9_remind_SSP5-Base_2030


Activity 'V1B_LTO_battery_final_battery_pack_perkWh_wh__remind_SSP5-Base_2040' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for LTO final battery pack for SSP _remind_SSP5-Base and year 2040, relying on BD ecoinvent_cutoff_3.9_remind_SSP5-Base_2040


Activity 'V1B_LTO_battery_final_battery_pack_perkWh_wh__remind_SSP5-Base_2050' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for LTO final battery pack for SSP _remind_SSP5-Base and year 2050, relying on BD ecoinvent_cutoff_3.9_remind_SSP5-Base_2050


## A.4 NCA::  final bat pack kWh
similar to LFP, battery pack available  "market for battery, Li-ion, NCA, rechargeable, prismatic"

In [12]:
for prm_name in ssp_scn:
    for year_t in ssp_yr: 
        bg_db_ssp_yr = bg_ei_ + prm_name + "_" + str(year_t)            # backgroun database (BD) to use for SSP[x], Year[t]
        copied_prm_UPR_suffix = "_from_" + prm_name + "_" + str(year_t) # copy UPR from BD to PDB for further use/modification if needed
        newUPR_suffix =  "_wh_" + prm_name + "_" + str(year_t)          # naming on any new UPRs created in PDB
        
        BAT_battery_pack = agb.newActivity(
                db_name=PDB_NAME,
                name = "V1B_NCA battery_final battery pack_perkWh"  +  newUPR_suffix, loc = 'CA-QC', 
                code = "V1B_NCA_battery_final_battery_pack_perkWh"  + newUPR_suffix, 
                unit="kWh",
                exchanges={
                    agb.findActivity("market for battery, Li-ion, NCA, rechargeable, prismatic", loc = "GLO", db_name = bg_db_ssp_yr): 1 *  (1/NCA_battery_cell_density )     
                },
        )
        
        for exc in BAT_battery_pack.getExchange(name = "*", single=False): 
            exc['type']='technosphere'
            exc.save()
        BAT_battery_pack.setOutputAmount(1)
                
        #agb.printAct(BAT_NMC_battery_pack)
        print(f"end of creating a new UPR for NCA final battery pack for SSP {prm_name} and year {year_t}, relying on BD {bg_db_ssp_yr}")
        

Activity 'V1B_NCA_battery_final_battery_pack_perkWh_wh__remind_SSP1-PkBudg500_2030' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for NCA final battery pack for SSP _remind_SSP1-PkBudg500 and year 2030, relying on BD ecoinvent_cutoff_3.9_remind_SSP1-PkBudg500_2030


Activity 'V1B_NCA_battery_final_battery_pack_perkWh_wh__remind_SSP1-PkBudg500_2040' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for NCA final battery pack for SSP _remind_SSP1-PkBudg500 and year 2040, relying on BD ecoinvent_cutoff_3.9_remind_SSP1-PkBudg500_2040


Activity 'V1B_NCA_battery_final_battery_pack_perkWh_wh__remind_SSP1-PkBudg500_2050' was already in 'iveo_Parameterized_v1'. Overwriting it


...

Activity 'V1B_NCA_battery_final_battery_pack_perkWh_wh__remind_SSP5-Base_2030' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for NCA final battery pack for SSP _remind_SSP5-Base and year 2030, relying on BD ecoinvent_cutoff_3.9_remind_SSP5-Base_2030


Activity 'V1B_NCA_battery_final_battery_pack_perkWh_wh__remind_SSP5-Base_2040' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for NCA final battery pack for SSP _remind_SSP5-Base and year 2040, relying on BD ecoinvent_cutoff_3.9_remind_SSP5-Base_2040
end of creating a new UPR for NCA final battery pack for SSP _remind_SSP5-Base and year 2050, relying on BD ecoinvent_cutoff_3.9_remind_SSP5-Base_2050


Activity 'V1B_NCA_battery_final_battery_pack_perkWh_wh__remind_SSP5-Base_2050' was already in 'iveo_Parameterized_v1'. Overwriting it


## A.5:: seperately  NMC622 / LFP / LTO / NCA EoL  kg -> per kWh by their specific energy  
### for running dynamic LCIA, can't mix battery pack with battery EoL into one UPR 
all LIBs using same BG_UPR, create new list adding for loop:  exchange :   -1/batden_p 

In [13]:
den_list = [NMC622_battery_cell_density, LFP_battery_cell_density, LTO_battery_cell_density, NCA_battery_cell_density]

In [14]:
name_list = ["NMC622", "LFP", "LTO", "NCA"]

In [15]:
for batx, batden_p in zip (name_list, den_list ): 

    for prm_name in ssp_scn:
        for year_t in ssp_yr: 
            bg_db_ssp_yr = bg_ei_ + prm_name + "_" + str(year_t)            # backgroun database (BD) to use for SSP[x], Year[t]
            copied_prm_UPR_suffix = "_from_" + prm_name + "_" + str(year_t) # copy UPR from BD to PDB for further use/modification if needed
            newUPR_suffix =  "_wh_" + prm_name + "_" + str(year_t)          # naming on any new UPRs created in PDB
            
            bat_EoL = agb.newActivity(
                    db_name = PDB_NAME,
                    name= "V1B_" + batx + "_EoL_perkWh_" + newUPR_suffix,
                    code = "V1B_" + batx + "_EoL_perkWh_" + newUPR_suffix,
                    unit="kWh",
                    exchanges={
                        agb.findActivity('market for used Li-ion battery', loc = 'GLO', db_name = bg_db_ssp_yr): (-1/batden_p ) ,
                    },
                )
            
            for exc in bat_EoL.getExchange(name = "*", single=False): 
                exc['type']='technosphere'
                exc.save()
            bat_EoL.setOutputAmount(1)
        
            print(f"end of creating a new UPR for EoL {batx} per kWh for SSP {prm_name} and year {year_t}, relying on BD {bg_db_ssp_yr}")
          

Activity 'V1B_NMC622_EoL_perkWh__wh__remind_SSP1-PkBudg500_2030' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for EoL NMC622 per kWh for SSP _remind_SSP1-PkBudg500 and year 2030, relying on BD ecoinvent_cutoff_3.9_remind_SSP1-PkBudg500_2030


Activity 'V1B_NMC622_EoL_perkWh__wh__remind_SSP1-PkBudg500_2040' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for EoL NMC622 per kWh for SSP _remind_SSP1-PkBudg500 and year 2040, relying on BD ecoinvent_cutoff_3.9_remind_SSP1-PkBudg500_2040


Activity 'V1B_NMC622_EoL_perkWh__wh__remind_SSP1-PkBudg500_2050' was already in 'iveo_Parameterized_v1'. Overwriting it


...

Activity 'V1B_NCA_EoL_perkWh__wh__remind_SSP5-Base_2030' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for EoL NCA per kWh for SSP _remind_SSP5-Base and year 2030, relying on BD ecoinvent_cutoff_3.9_remind_SSP5-Base_2030


Activity 'V1B_NCA_EoL_perkWh__wh__remind_SSP5-Base_2040' was already in 'iveo_Parameterized_v1'. Overwriting it


end of creating a new UPR for EoL NCA per kWh for SSP _remind_SSP5-Base and year 2040, relying on BD ecoinvent_cutoff_3.9_remind_SSP5-Base_2040
end of creating a new UPR for EoL NCA per kWh for SSP _remind_SSP5-Base and year 2050, relying on BD ecoinvent_cutoff_3.9_remind_SSP5-Base_2050


Activity 'V1B_NCA_EoL_perkWh__wh__remind_SSP5-Base_2050' was already in 'iveo_Parameterized_v1'. Overwriting it


# B. calculate LIBs UPRs per FU kWh under different SSP with yearly P

#### B.0 test opening Parameter file

In [10]:
for nmc_final in NMC_final_allsspyear:
    print(nmc_final, "~~~~~~~~~~~~")

'V1B_NMC622 battery_final battery pack_perkWh_wh__remind_SSP1-PkBudg500_2030' (kWh, GLO, None) ~~~~~~~~~~~~
'V1B_NMC622 battery_final battery pack_perkWh_wh__remind_SSP1-PkBudg500_2040' (kWh, GLO, None) ~~~~~~~~~~~~
'V1B_NMC622 battery_final battery pack_perkWh_wh__remind_SSP1-PkBudg500_2050' (kWh, GLO, None) ~~~~~~~~~~~~
'V1B_NMC622 battery_final battery pack_perkWh_wh__remind_SSP1-PkBudg1150_2030' (kWh, GLO, None) ~~~~~~~~~~~~
'V1B_NMC622 battery_final battery pack_perkWh_wh__remind_SSP1-PkBudg1150_2040' (kWh, GLO, None) ~~~~~~~~~~~~
'V1B_NMC622 battery_final battery pack_perkWh_wh__remind_SSP1-PkBudg1150_2050' (kWh, GLO, None) ~~~~~~~~~~~~
'V1B_NMC622 battery_final battery pack_perkWh_wh__remind_SSP2-Base_2030' (kWh, GLO, None) ~~~~~~~~~~~~
'V1B_NMC622 battery_final battery pack_perkWh_wh__remind_SSP2-Base_2040' (kWh, GLO, None) ~~~~~~~~~~~~
'V1B_NMC622 battery_final battery pack_perkWh_wh__remind_SSP2-Base_2050' (kWh, GLO, None) ~~~~~~~~~~~~
'V1B_NMC622 battery_final battery pack_p

In [11]:
root_path = '/Users/susierwu/Dropbox/DLCA_DynCO2/pdpLCA_GT/'
p_file = os.path.abspath(os.path.join(root_path, 'data/P_files/p_tech_perSSP_Y.xlsx'))

for nmc_final in NMC_final_allsspyear:
    # need a matched YearlyParams for each SSPx:
    str1 = nmc_final.as_dict()["name"]
    str2 = str1.split("remind_")[1]
    year_to_get_inP = int( str2.split("_")[-1] )
    ssp_remind = str2.split("_")[0] 
    ssp_to_get_inP = ssp_remind_Pname_map.get(ssp_remind)
    
    yearlyParams = loadYearlyParams(PARAMS_FILE = p_file, s_name = "V1B_bat_prmLIBs_perkWh", years = [year_to_get_inP], SSP = ssp_to_get_inP) 
    print(f"for SSP {ssp_to_get_inP}, year {year_to_get_inP}, P:  {yearlyParams}" )

for SSP ssp119, year 2030, P:  {'NMC622_battery_cell_density': [0.29161000000000004], 'NCA_battery_cell_density': [0.30358900000000005], 'LFP_battery_cell_density': [0.21054000000000003], 'LTO_battery_cell_density': [0.21054000000000003]}
for SSP ssp119, year 2040, P:  {'NMC622_battery_cell_density': [0.3528481000000001], 'NCA_battery_cell_density': [0.36734269000000014], 'LFP_battery_cell_density': [0.2547534000000001], 'LTO_battery_cell_density': [0.2547534000000001]}
for SSP ssp119, year 2050, P:  {'NMC622_battery_cell_density': [0.4269462010000002], 'NCA_battery_cell_density': [0.4444846549000002], 'LFP_battery_cell_density': [0.3082516140000001], 'LTO_battery_cell_density': [0.3082516140000001]}
for SSP ssp126, year 2030, P:  {'NMC622_battery_cell_density': [0.3187224999999999], 'NCA_battery_cell_density': [0.33181524999999995], 'LFP_battery_cell_density': [0.23011499999999996], 'LTO_battery_cell_density': [0.23011499999999996]}
for SSP ssp126, year 2040, P:  {'NMC622_battery_cell

### B.1  read in NMC parameter under different SSPx in the calc


In [12]:
#final output to be appended
nmc_lcia_dict = {}
GWP1_list, GWP2_list, SSP_list, Year_list = [],[],[],[]

for nmc_final in NMC_final_allsspyear:
    # need a matched YearlyParams for each SSPx:
    str1 = nmc_final.as_dict()["name"]
    str2 = str1.split("remind_")[1]
    year_to_get_inP = int( str2.split("_")[-1] )
    ssp_remind = str2.split("_")[0] 
    ssp_to_get_inP = ssp_remind_Pname_map.get(ssp_remind)
    
    yearlyParams = loadYearlyParams(PARAMS_FILE = p_file, s_name = "V1B_bat_prmLIBs_perkWh", years = [year_to_get_inP], SSP = ssp_to_get_inP) 
    
    df = agb.multiLCAAlgebric(models=[
                (nmc_final, 1)], 
                methods = chosen_methods,   
                **yearlyParams
                )
            
    print(f"end of running LCA calc for SSP {ssp_to_get_inP} and year {year_to_get_inP} for {nmc_final} ")
    #display(df)
    
    # prepare for final DF: 
    GWP1 = df.iloc[:,0].values[0]
    GWP2 = df.iloc[:,1].values[0]
    GWP1_list.append(GWP1)
    GWP2_list.append(GWP2)
    SSP_list.append(ssp_to_get_inP) 
    Year_list.append(year_to_get_inP)
    
    # assign df as value to each UPR name (used as key)
    nmc_lcia_dict[str1] = df

end of running LCA calc for SSP ssp119 and year 2030 for 'V1B_NMC622 battery_final battery pack_perkWh_wh__remind_SSP1-PkBudg500_2030' (kWh, GLO, None) 
end of running LCA calc for SSP ssp119 and year 2040 for 'V1B_NMC622 battery_final battery pack_perkWh_wh__remind_SSP1-PkBudg500_2040' (kWh, GLO, None) 
end of running LCA calc for SSP ssp119 and year 2050 for 'V1B_NMC622 battery_final battery pack_perkWh_wh__remind_SSP1-PkBudg500_2050' (kWh, GLO, None) 
end of running LCA calc for SSP ssp126 and year 2030 for 'V1B_NMC622 battery_final battery pack_perkWh_wh__remind_SSP1-PkBudg1150_2030' (kWh, GLO, None) 
end of running LCA calc for SSP ssp126 and year 2040 for 'V1B_NMC622 battery_final battery pack_perkWh_wh__remind_SSP1-PkBudg1150_2040' (kWh, GLO, None) 
end of running LCA calc for SSP ssp126 and year 2050 for 'V1B_NMC622 battery_final battery pack_perkWh_wh__remind_SSP1-PkBudg1150_2050' (kWh, GLO, None) 
end of running LCA calc for SSP ssp245 and year 2030 for 'V1B_NMC622 battery_fi

In [13]:
lcia_nmc622 = pd.DataFrame(zip(SSP_list, Year_list, GWP1_list, GWP2_list), columns = ["SSP","Year","GWPkgCO2e_perkWhNMC622", "prmGWPkgCO2e_perkWhNMC622"])
lcia_nmc622

,SSP,Year,GWPkgCO2e_perkWhNMC622,prmGWPkgCO2e_perkWhNMC622
0,ssp119,2030,41.584126,42.533805
1,ssp119,2040,29.199205,29.431573
2,ssp119,2050,23.047449,22.571552
3,ssp126,2030,43.788512,44.769287
4,ssp126,2040,27.879297,28.674196
5,ssp126,2050,18.632135,19.215818
6,ssp245,2030,50.328276,51.393080
7,ssp245,2040,37.101369,38.065880
8,ssp245,2050,27.893087,28.743821
9,ssp585,2030,58.182930,59.315939
